This is the first attempt sequence....

In [14]:
%matplotlib inline

import healpy as hp
import numpy as np
import json
import astropy.coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
from astropy.io import fits
import csv
from scipy.special import erf
import astroquery as ast


In [15]:
#The following function finds distance between two points, and returns whether or not the points are within distance r of each other
#r is the diagonal range available within one field of view of the telescope in question.

def within_field_check(r,ra1,ra2,dec1,dec2):
    
    one_field = [] #this forms a list of galaxies within one field of view
    dist = np.sqrt((ra2*cos(dec2)-ra1*cos(dec1))**2. + (dec2-dec1)**2.)
    if dist<r:
        one_field.append(dist)
    else:
        return ("Cannot fit in frame")

In [17]:
with open("Thacher_Galaxies_sorted_by_dist.csv","rU") as csvinput:
        
    reader = csv.reader(csvinput, delimiter=",")

    r = 3. #arbitrary. What is Thacher's field of view?
        
    for row in reader:
            
        if used == False:
            
            ra1 = np.cos(row[2])*row[1] #RA requires a cosine factor.
            dec1 = row[2]
        
            ra2 = np.cos((row+1)[2])*(row+1)[1]#RA requires a cosine factor.                dec2 = (row+1)[2]
        
            one_field = [] #Forms an empty list to put galaxies within one field of view in
            
            dist = np.sqrt(dist = np.sqrt((ra2-ra1)**2. + (dec2-dec1)**2.))              
            
            if dist<r:
                    one_field.append(dist)
                    print (one_field)
            else:
                print ("Cannot fit in frame")
                    
            used == True

        if used ==True:

            row = row+1 #We need to skip the ones which have already been chosen
        

/Users/dragon/anaconda/envs/astroconda_35/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: 'U' mode is deprecated
  if __name__ == '__main__':


NameError: name 'used' is not defined

This is what I had in C++
#include <iostream>
#include <cmath>

int main(int argc, const char * argv[])
{
	FILE* myFile = fopen("Thacher_nameradec_sorteddist.csv", "r");
	
	
	const int nElemsMax = 10;
	
	char name[nElemsMax][32];
	double ra[nElemsMax], dec[nElemsMax];
	
	int nItems(0);
	for( int i = 0; i < nElemsMax; i++ )
	{
		int got = fscanf(myFile, "%32[^,], %lf, %lf", name[i], &ra[i], &dec[i]);
		if(got != 3) break;
		if(name[i][0] == '\n')
			strcpy(name[i], &name[i][1]);
		
		printf("Name: %s,  RA[%d] = %lf, Dec[%d] = %lf\n", name[i], i, ra[i], i, dec[i]);
		nItems++;
	}

	fclose(myFile);
	
	
	FILE* output = fopen("outpute_test.csv","w+");
	
	bool used[nElemsMax];
	
	for( int i = 0; i < nItems; i++ )
		used[i]=false;
	
	double distLimit_arcmin = 3.0;
	for( int i = 0; i < nItems; i++ )
	{
		if( !used[i] )
		{
			double this_ra = ra[i];
			double this_dec = dec[i];
		
			printf("For Galaxy: %s, RA: %lf,  Dec: %lf\n", name[i], ra[i], dec[i]);
			fprintf(output, "For Galaxy: %s, RA: %lf,  Dec: %lf\n", name[i], ra[i], dec[i]);
			used[i]=true;
		
			for(int j = 0; j < nItems; j++)
			{
				if( j != i && ! used[j])
				{
					double dist = std::sqrt( std::pow(ra[j]-this_ra, 2) + std::pow(dec[j]-this_dec, 2) );
					if( dist < distLimit_arcmin )
					{
						printf("Name: %s, RA: %lf,  Dec: %lf\n", name[j], ra[j], dec[j]);
						fprintf(output, "Name: %s, RA: %lf,  Dec: %lf\n", name[j], ra[j], dec[j]);
						used[j]=true;
					
					}
				}
			}
		}
	}
	fclose(output);
	return 0;
}

In [ ]:
with open("Thacher_Galaxies_sorted_by_dist.csv","rU") as csvinput:
    with open("Thacher_Galaxies_within_field_query.csv","w") as csvoutput:
        
        reader = csv.reader(csvinput, delimiter=",")
        writer = csv.writer(csvoutput, lineterminator="\n")
        
        output_rows = []
        header_row = next(reader)
        header_row.append("Within_field")
        output_rows.append(header_row)
        
# Better estimate for RA 0.2010cos3ωtdegrees

        Within_field = []

        for row in reader:
            Name = row[0]
            ra = row[1]*np.cos(row[1]) # RA (degree) col
            dec = row[2] # DEC (degree) col
            c = coord.SkyCoord(ra+dec,unit=(u.hourangle, u.deg),frame="fk5")
            
            ra2 = reader(i+1)*np.cos(i+1)
            dec2 = row[2]
            one_field = [] #this forms a list of galaxies within one field of view
            dist = np.sqrt((ra2-ra1)**2. + (dec2-dec1)**2.)
            if dist<r:
                one_field.append(dist)
            else:
                print ("Cannot fit in frame")
            
    
            row.append(Within_field)
            output_rows.append(row)

        writer.writerows(output_rows)

In [6]:
#Both of these functions below take in N numbers, and 
#find the distance between each number and every other number.
#The second one is faster, if we care.


def dist_array_easy(U):
    Uarray = np.arange(U ** 2)
    Ui = np.floor(Uarray / U)
    di = np.abs(Ui.reshape(-1, 1) - Ui)
    di = np.minimum(di, U - di)
    Uj = Ui * U - Uarray
    dj = np.abs(Uarray.reshape(-1, 1) - Ui.reshape(-1, 1) * U + Uj)
    dj = np.minimum(dj, U - dj)
    return np.hypot(di, dj)

def dist_array(U):
    def d(i):
        i = np.abs(i.reshape(-1, 1) - i.reshape(-1))
        return np.minimum(i, U - i)
    i, j = np.mgrid[:U, :U]
    return np.hypot(d(i), d(j))